In [127]:
import pickle
import nltk
from nltk.corpus import stopwords
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import pandas as pd

In [ ]:
nltk.download() # follow the prompts to download "stopwords"
# stopwords.words('english')

In [194]:
# Create bag o words feature vector
stops = stopwords.words('english')
def text_preprocess(text:str):
    # Ignoring case
    text = text.lower()
    # Ignoring punctuation
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Ignoring frequent words that don’t contain much information, called stop words, like “a,” “of,” etc.
    text = text.split(' ')
    text = [word for word in text if not word in stops]
    text = ' '.join(text)
    return text


def aggregate_features(x):
    return [*x['word_bag'], *x['bill_type_vec'], *x['control_vec'], *x['pass_law_vec']]


def one_hot_encoder(data:list):
    n_types = len(set(data))
    type_key = list(set(data))
    encoded = [None]*len(data)
    for i, x in enumerate(data):
        vec = [0]*n_types
        index = type_key.index(x)
        vec[index] = 1
        encoded[i] = vec
    return encoded

In [196]:
bill_features = pd.read_csv('/home/sam/neu/gml/data/congress/original/bills_features_label.csv')
bill_features = bill_features
clean_title_text = bill_features['title_text'].apply(text_preprocess)
# Initialize the "CountVectorizer" object, which is scikit-learn's bag of words tool.
vectorizer = CountVectorizer(analyzer = 'word',
                            tokenizer = None,
                            preprocessor = None,
                            stop_words = None,
                            max_features = 118)
# fit_transform() does two functions: First, it fits the model
# and learns the vocaulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of strings.
train_data_features = vectorizer.fit_transform(clean_title_text)

# Numpy arrays are easy to work with, so convert the result to an array
train_data_features = train_data_features.toarray()
train_data_features_normed = normalize(train_data_features)
print('Bag of words completed')
bill_features['word_bag'] = list(train_data_features_normed)
#  add party_control to the feature vector
bill_features['control_vec'] = bill_features['party_control_congress'].apply(
                                                    lambda x: [x/100-1])
bill_features['bill_type_vec'] = one_hot_encoder(list(bill_features['bill_type']))
bill_features['pass_law_vec'] = bill_features['pass_law'].apply(lambda x: [x])
bill_features['vector'] = bill_features.apply(aggregate_features, axis=1)

Bag of words completed


In [198]:
df_features = bill_features[['bill_id', 'vector']]
with open('/home/sam/neu/gml/data/congress/original/features_df.csv', 'wb') as f:
    df_features.to_csv(f)

In [205]:
mm_features = np.load('/home/sam/neu/gml/G-Meta/G-Meta_Data/FirstMM_DB/features.npy', allow_pickle=True)
mm_features[0].dtype

dtype('float32')